# 如何使用預訓練的模型來分類照片中的物體

卷積神經網絡現在能夠在一些電腦視覺任務上勝過人類的肉眼，例如圖像分類。

也就是說，給出一個物體的照片，我們可以讓電腦來回答這個照片是1000個特定
種類物體中的哪一類這樣的問題。

在本教程中，我們將使用幾個Keras己經內建好的預訓練模型來進行圖像分類, 其中包括了:
* VGG16
* VGG19
* ResNet50
* InceptionV3
* InceptionResNetV2
* Xception
* MobileNet

![imagenet](https://kaggle2.blob.core.windows.net/datasets-images/2798/4640/d21dc5df65889e105877476d81240e11/data-original.png)

In [ ]:
# 這個Jupyter Notebook的環境
import platform
import tensorflow
import keras
print("Platform: {}".format(platform.platform()))
print("Tensorflow version: {}".format(tensorflow.__version__))
print("Keras version: {}".format(keras.__version__))

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from IPython.display import Image

## 開發一個簡單的照片分類器

### VGG16

### 1.獲取範例圖像


首先，我們需要一個我們可以進行分類的圖像。

您可以在這裡從Google隨意檢索一些動物照片並下載到這個jupyter notebook所處的目錄。比如說:
我下載了: https://www.elephantvoices.org/images/slider/evimg16tf.jpg

![evimg16tf.jpg](https://www.elephantvoices.org/images/slider/evimg16tf.jpg)


### 2.加載VGG模型

加載在Keras己經預訓練好的VGG-16的權重模型檔。

In [2]:
from keras.applications.vgg16 import VGG16

# 載入權重
model_vgg16 = VGG16()

### 3.加載和準備圖像

接下來，我們可以將圖像加載進來，並轉換成預訓練網絡所要求的張量規格。

Keras提供了一些工具來幫助完成這一步驟。

首先，我們可以使用load_img（）函數加載圖像，並將其大小調整為224×224像素所需的大小。

In [4]:
from keras.preprocessing.image import load_img

# 載入圖像檔
img_file = 'evimg16tf.jpg'
image = load_img(img_file, target_size=(224, 224)) # 因為VGG16的模型的輸入是224x224

接下來，我們可以將像素轉換為NumPy數組，以便我們可以在Keras中使用它。我們可以使用這個img_to_array（）函數。

In [5]:
from keras.preprocessing.image import img_to_array

# 將圖像資料轉為numpy array
image = img_to_array(image) # RGB

print("image.shape:", image.shape)

image.shape: (224, 224, 3)


VGG16網絡期望單色階(grey)或多色階圖像(rgb)來作為輸入;這意味著輸入數組需要是轉換成四個維度：

(圖像批次量，圖像高，圖像寬, 圖像色階數) -> (batch_size, img_height, img_width, img_channels)

我們只有一個樣本（一個圖像）。我們可以通過調用reshape（）來重新調整數組的形狀，並添加額外的維度。

In [6]:
# 調整張量的維度
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

print("image.shape:", image.shape)

image.shape: (1, 224, 224, 3)


接下來，我們需要按照VGG在訓練ImageNet數據一樣的方法來對圖像進行前處理。具體來說，從論文裡談到：
> The only preprocessing we do is subtracting the mean RGB value, computed on the training set, from each pixel.
>
> __Very Deep Convolutional Networks for Large-Scale Image Recognition, 2014.__

Keras提供了一個稱為preprocess_input（）的函數來為VGG網絡準備新的圖像輸入。


In [7]:
from keras.applications.vgg16 import preprocess_input

# 準備VGG模型的圖像
image = preprocess_input(image)

### 4.做一個預測

我們可以調用模型中的predict（）函數來預測圖像屬於1000個已知對像類型的機率。

In [8]:
# 預測所有產出類別的機率

y_pred = model_vgg16.predict(image)

In [10]:
y_pred.shape

(1, 1000)

### 5.解釋預測

Keras提供了一個函數來解釋稱為decode_predictions（）的概率。

它可以返回一個類別的列表和每一個類別機率，為了簡單起見, 我們只會秀出第一個機率最高的種類。


In [11]:
from keras.applications.vgg16 import decode_predictions

# 將機率轉換為類別標籤
label = decode_predictions(y_pred)

# 檢索最可能的結果，例如最高的概率
label = label[0][0]

# 打印預測結果
print('%s (%.2f%%)' % (label[1], label[2]*100))

African_elephant (61.56%)


In [28]:
model_vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### VGG19

In [12]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions

# 載入權重
model_vgg19 = VGG19(weights='imagenet')
# 載入圖像檔
img_file = 'evimg16tf.jpg'
image = load_img(img_file, target_size=(224, 224)) # 因為VGG19的模型的輸入是224x224

# 將圖像資料轉為numpy array
image = img_to_array(image) # RGB
print("image.shape:", image.shape)

# 調整張量的維度
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print("image.shape:", image.shape)

# 準備模型所需要的圖像前處理
image = preprocess_input(image)

# 預測所有產出類別的機率
y_pred = model_vgg19.predict(image)

# 將機率轉換為類別標籤
label = decode_predictions(y_pred)

# 檢索最可能的結果，例如最高的概率
label = label[0][0]

# 打印預測結果
print('%s (%.2f%%)' % (label[1], label[2]*100))

image.shape: (224, 224, 3)
image.shape: (1, 224, 224, 3)
tusker (49.97%)


In [27]:
model_vgg19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### ResNet50

In [13]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import decode_predictions

# 載入權重
model_resnet50 = ResNet50(weights='imagenet')

# 載入圖像檔
img_file = 'evimg16tf.jpg'

# 因為ResNet的模型的輸入是224x224
image = load_img(img_file, target_size=(224, 224)) 

# 將圖像資料轉為numpy array
image = img_to_array(image) # RGB
print("image.shape:", image.shape)

# 調整張量的維度
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print("image.shape:", image.shape)

# 準備模型所需要的圖像前處理
image = preprocess_input(image)

# 預測所有產出類別的機率
y_pred = model_resnet50.predict(image)

# 將機率轉換為類別標籤
label = decode_predictions(y_pred)

# 檢索最可能的結果，例如最高的概率
label = label[0][0]

# 打印預測結果
print('%s (%.2f%%)' % (label[1], label[2]*100))

image.shape: (224, 224, 3)
image.shape: (1, 224, 224, 3)
tusker (41.42%)


In [26]:
model_resnet50.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

### InceptionV3

In [20]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions

# 載入權重
model_inception_v3 = InceptionV3(weights='imagenet')

# 載入圖像檔
img_file = 'evimg16tf.jpg'
# InceptionV3的模型的輸入是299x299
img = load_img(img_file, target_size=(299, 299)) 

# 將圖像資料轉為numpy array
image = image.img_to_array(img) # RGB
print("image.shape:", image.shape)

# 調整張量的維度
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print("image.shape:", image.shape)

# 準備模型所需要的圖像前處理
image = preprocess_input(image)

# 預測所有產出類別的機率
y_pred = model_inception_v3.predict(image)

# 將機率轉換為類別標籤
label = decode_predictions(y_pred)

# 檢索最可能的結果，例如最高的概率
label = label[0][0]

# 打印預測結果
print('%s (%.2f%%)' % (label[1], label[2]*100))

image.shape: (299, 299, 3)
image.shape: (1, 299, 299, 3)
tusker (44.79%)


In [25]:
model_inception_v3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_392 (Conv2D)             (None, None, None, 3 864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_392 (BatchN (None, None, None, 3 96          conv2d_392[0][0]                 
__________________________________________________________________________________________________
activation_441 (Activation)     (None, None, None, 3 0           batch_normalization_392[0][0]    
__________________________________________________________________________________________________
conv2d_393

### InceptionResNetV2

In [21]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.applications.inception_resnet_v2 import decode_predictions

# 載入權重
model_inception_resnet_v2 = InceptionResNetV2(weights='imagenet')

# 載入圖像檔
img_file = 'evimg16tf.jpg'

# InceptionResNetV2的模型的輸入是299x299
image = load_img(img_file, target_size=(299, 299)) 

# 將圖像資料轉為numpy array
image = img_to_array(image) # RGB
print("image.shape:", image.shape)

# 調整張量的維度
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print("image.shape:", image.shape)

# 準備模型所需要的圖像前處理
image = preprocess_input(image)

# 預測所有產出類別的機率
y_pred = model_inception_resnet_v2.predict(image)

# 將機率轉換為類別標籤
label = decode_predictions(y_pred)

# 檢索最可能的結果，例如最高的概率
label = label[0][0]

# 打印預測結果
print('%s (%.2f%%)' % (label[1], label[2]*100))

image.shape: (299, 299, 3)
image.shape: (1, 299, 299, 3)
tusker (58.08%)


In [22]:
model_inception_resnet_v2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_486 (Conv2D)             (None, None, None, 3 864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_486 (BatchN (None, None, None, 3 96          conv2d_486[0][0]                 
__________________________________________________________________________________________________
activation_535 (Activation)     (None, None, None, 3 0           batch_normalization_486[0][0]    
__________________________________________________________________________________________________
conv2d_487

__________________________________________________________________________________________________
activation_695 (Activation)     (None, None, None, 2 0           batch_normalization_646[0][0]    
__________________________________________________________________________________________________
conv2d_642 (Conv2D)             (None, None, None, 2 278528      block17_20_ac[0][0]              
__________________________________________________________________________________________________
conv2d_644 (Conv2D)             (None, None, None, 2 278528      block17_20_ac[0][0]              
__________________________________________________________________________________________________
conv2d_647 (Conv2D)             (None, None, None, 2 663552      activation_695[0][0]             
__________________________________________________________________________________________________
batch_normalization_642 (BatchN (None, None, None, 2 768         conv2d_642[0][0]                 
__________

### MobileNet

In [23]:
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.applications.mobilenet import decode_predictions

# 載入權重
model_mobilenet = MobileNet(weights='imagenet')

# 載入圖像檔
img_file = 'evimg16tf.jpg'

# MobileNet的模型的輸入是224x224
image = load_img(img_file, target_size=(224, 224)) 

# 將圖像資料轉為numpy array
image = img_to_array(image) # RGB
print("image.shape:", image.shape)

# 調整張量的維度
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print("image.shape:", image.shape)

# 準備模型所需要的圖像前處理
image = preprocess_input(image)

# 預測所有產出類別的機率
y_pred = model_mobilenet.predict(image)

# 將機率轉換為類別標籤
label = decode_predictions(y_pred)

# 檢索最可能的結果，例如最高的概率
label = label[0][0]

# 打印預測結果
print('%s (%.2f%%)' % (label[1], label[2]*100))

image.shape: (224, 224, 3)
image.shape: (1, 224, 224, 3)
African_elephant (52.44%)


In [24]:
model_mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

## 總結 (Conclusion)

在這篇文章中有一些個人學習到的一些有趣的重點:
    
* 在Keras中己經預建了許多高級的圖像識別的網絡及預訓練的權重
* 需要了解每種高級圖像識別的網絡的結構與輸入的張量
* 了解不同高級圖像識別的網絡的訓練變數量與預訓練的權重可以有效幫助圖像識別類型的任務

參考: 
* [How to Use The Pre-Trained VGG Model to Classify Objects in Photographs](https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/)
* [Keras官網](http://keras.io/)